In [ ]:
# ============================================================
# TIER 2: Mango Leaf Damage + Severity Estimator
# ============================================================

import cv2
import numpy as np
import pandas as pd
from pathlib import Path

# -------- SETTINGS --------
image_dir = "../../Database/dataset/Anthracnose/" 
output_csv = "leaf_features_tier2.csv"
cm_per_pixel = 0.01   # calibrate using a known reference

# -------- UTILITIES --------
def area_cm2(pixels, cm_per_pixel):
    return pixels * (cm_per_pixel ** 2)

def segment_leaf(img_hsv):
    lower = np.array([20, 40, 40])
    upper = np.array([90, 255, 255])
    mask = cv2.inRange(img_hsv, lower, upper)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((5,5),np.uint8))
    return mask

def segment_lesion(img_hsv, leaf_mask):
    lower = np.array([5, 30, 0])
    upper = np.array([25, 255, 160])
    mask = cv2.inRange(img_hsv, lower, upper)
    mask = cv2.bitwise_and(mask, leaf_mask)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3,3),np.uint8))
    return mask

# -------- MAIN LOOP --------
records = []

for path in Path(image_dir).glob("*.jpg"):
    img = cv2.imread(str(path))
    if img is None:
        continue

    img = cv2.resize(img, (800, int(img.shape[0]*800/img.shape[1])))
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # --- segmentation ---
    leaf_mask = segment_leaf(hsv)
    lesion_mask = segment_lesion(hsv, leaf_mask)
    contours, _ = cv2.findContours(leaf_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        continue
    largest_contour = max(contours, key=cv2.contourArea)

    # --- base areas ---
    leaf_area_px = cv2.countNonZero(leaf_mask)
    lesion_area_px = cv2.countNonZero(lesion_mask)
    leaf_area_cm2 = area_cm2(leaf_area_px, cm_per_pixel)
    lesion_area_cm2 = area_cm2(lesion_area_px, cm_per_pixel)

    severity_pct = (lesion_area_px / leaf_area_px * 100) if leaf_area_px > 0 else 0

    # --- reconstruction: convex hull ---
    hull = cv2.convexHull(largest_contour)
    hull_area = cv2.contourArea(hull)
    hull_area_cm2 = area_cm2(hull_area, cm_per_pixel)
    missing_pct_hull = max((hull_area - leaf_area_px) / hull_area * 100, 0)

    # --- reconstruction: ellipse fit (if possible) ---
    ellipse_area_px = None
    missing_pct_ellipse = None
    if len(largest_contour) >= 5:
        ellipse = cv2.fitEllipse(largest_contour)
        ellipse_area_px = np.pi * (ellipse[1][0]/2) * (ellipse[1][1]/2)
        missing_pct_ellipse = max((ellipse_area_px - leaf_area_px) / ellipse_area_px * 100, 0)

    # --- color stats ---
    leaf_mean = cv2.mean(img, mask=leaf_mask)[:3]
    lesion_mean = cv2.mean(img, mask=lesion_mask)[:3]

    # --- save record ---
    records.append({
        "filename": path.name,
        "leaf_area_px": leaf_area_px,
        "lesion_area_px": lesion_area_px,
        "leaf_area_cm2": leaf_area_cm2,
        "lesion_area_cm2": lesion_area_cm2,
        "severity_pct": severity_pct,
        "hull_area_px": hull_area,
        "hull_missing_pct": missing_pct_hull,
        "ellipse_area_px": ellipse_area_px,
        "ellipse_missing_pct": missing_pct_ellipse,
        "leaf_mean_B": leaf_mean[0],
        "leaf_mean_G": leaf_mean[1],
        "leaf_mean_R": leaf_mean[2],
        "lesion_mean_B": lesion_mean[0],
        "lesion_mean_G": lesion_mean[1],
        "lesion_mean_R": lesion_mean[2]
    })

    # --- visualization ---
    overlay = img.copy()
    overlay[leaf_mask == 0] = (0, 0, 0)
    cv2.drawContours(overlay, [hull], -1, (255, 0, 0), 2)  # blue hull
    cv2.drawContours(overlay, cv2.findContours(lesion_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0], -1, (0, 0, 255), 2)
    cv2.putText(overlay, f"Severity: {severity_pct:.1f}%  Missing: {missing_pct_hull:.1f}%", (20,40),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
    cv2.imshow("Leaf Analysis", overlay)
    cv2.waitKey(200)

cv2.destroyAllWindows()

# -------- SAVE --------
df = pd.DataFrame(records)
df.to_csv(output_csv, index=False)
print("✅ Tier 2 extraction complete.")
print(df.head())
# ============================================================

✅ Tier 2 extraction complete.
                       filename  leaf_area_px  lesion_area_px  leaf_area_cm2  \
0  20211008_124249 (Custom).jpg        188483           76080        18.8483   
1  20211008_124250 (Custom).jpg        176272           66566        17.6272   
2  20211008_124252 (Custom).jpg        198491           70074        19.8491   
3  20211008_124253 (Custom).jpg        175075           60201        17.5075   
4  20211008_124256 (Custom).jpg        227100          119754        22.7100   

   lesion_area_cm2  severity_pct  hull_area_px  hull_missing_pct  \
0           7.6080     40.364383      199510.0          5.527041   
1           6.6566     37.763230      186933.0          5.703113   
2           7.0074     35.303364      210189.5          5.565692   
3           6.0201     34.385835      188495.0          7.119552   
4          11.9754     52.731836      248839.5          8.736354   

   ellipse_area_px  ellipse_missing_pct  leaf_mean_B  leaf_mean_G  \
0    427116

In [2]:
# ============================================================
# TIER 2: Mango Leaf Damage + Severity Estimator (with Inpainting)
# ============================================================

import cv2
import numpy as np
import pandas as pd
from pathlib import Path

# -------- SETTINGS --------
image_dir = "../../database/dataset/Anthracnose/" 
output_csv = "leaf_features_tier2.csv"
cm_per_pixel = 0.01   # calibrate using a known reference

# -------- UTILITIES --------
def area_cm2(pixels, cm_per_pixel):
    return pixels * (cm_per_pixel ** 2)

def segment_leaf(img_hsv):
    lower = np.array([20, 40, 40])
    upper = np.array([90, 255, 255])
    mask = cv2.inRange(img_hsv, lower, upper)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((5,5), np.uint8))
    return mask

def segment_lesion(img_hsv, leaf_mask):
    lower = np.array([5, 30, 0])
    upper = np.array([25, 255, 160])
    mask = cv2.inRange(img_hsv, lower, upper)
    mask = cv2.bitwise_and(mask, leaf_mask)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
    return mask

def reconstruct_leaf_inpaint(img, leaf_mask):
    """Use OpenCV inpainting to reconstruct missing/damaged leaf areas."""
    # Find holes inside the leaf mask (invert)
    holes = cv2.bitwise_not(leaf_mask)
    holes = cv2.morphologyEx(holes, cv2.MORPH_OPEN, np.ones((5,5), np.uint8))
    
    # Create inpainting mask only inside convex hull of leaf
    contours, _ = cv2.findContours(leaf_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return leaf_mask, 0, img
    
    largest_contour = max(contours, key=cv2.contourArea)
    hull = cv2.convexHull(largest_contour)
    hull_mask = np.zeros_like(leaf_mask)
    cv2.drawContours(hull_mask, [hull], -1, 255, -1)
    inpaint_mask = cv2.bitwise_and(holes, hull_mask)

    # Estimate missing percentage (how much of hull is empty)
    missing_px = cv2.countNonZero(inpaint_mask)
    hull_area = cv2.countNonZero(hull_mask)
    damage_pct_inpaint = (missing_px / hull_area * 100) if hull_area > 0 else 0

    # Perform inpainting on original image for visualization
    inpainted_img = cv2.inpaint(img, inpaint_mask, 5, cv2.INPAINT_TELEA)

    # Reconstruct leaf mask (fill holes)
    reconstructed_mask = cv2.bitwise_or(leaf_mask, inpaint_mask)

    return reconstructed_mask, damage_pct_inpaint, inpainted_img

# -------- MAIN LOOP --------
records = []

for path in Path(image_dir).glob("*.jpg"):
    img = cv2.imread(str(path))
    if img is None:
        continue

    img = cv2.resize(img, (800, int(img.shape[0]*800/img.shape[1])))
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # --- segmentation ---
    leaf_mask = segment_leaf(hsv)
    lesion_mask = segment_lesion(hsv, leaf_mask)

    # --- reconstruction (inpainting) ---
    leaf_mask_recon, damage_pct_inpaint, inpainted_img = reconstruct_leaf_inpaint(img, leaf_mask)

    contours, _ = cv2.findContours(leaf_mask_recon, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        continue
    largest_contour = max(contours, key=cv2.contourArea)

    # --- base areas ---
    leaf_area_px = cv2.countNonZero(leaf_mask_recon)
    lesion_area_px = cv2.countNonZero(lesion_mask)
    leaf_area_cm2 = area_cm2(leaf_area_px, cm_per_pixel)
    lesion_area_cm2 = area_cm2(lesion_area_px, cm_per_pixel)
    severity_pct = (lesion_area_px / leaf_area_px * 100) if leaf_area_px > 0 else 0

    # --- convex hull + ellipse reconstruction ---
    hull = cv2.convexHull(largest_contour)
    hull_area = cv2.contourArea(hull)
    hull_area_cm2 = area_cm2(hull_area, cm_per_pixel)
    missing_pct_hull = max((hull_area - leaf_area_px) / hull_area * 100, 0)

    ellipse_area_px = None
    missing_pct_ellipse = None
    if len(largest_contour) >= 5:
        ellipse = cv2.fitEllipse(largest_contour)
        ellipse_area_px = np.pi * (ellipse[1][0]/2) * (ellipse[1][1]/2)
        missing_pct_ellipse = max((ellipse_area_px - leaf_area_px) / ellipse_area_px * 100, 0)

    # --- color stats ---
    leaf_mean = cv2.mean(img, mask=leaf_mask_recon)[:3]
    lesion_mean = cv2.mean(img, mask=lesion_mask)[:3]

    # --- save record ---
    records.append({
        "filename": path.name,
        "leaf_area_px": leaf_area_px,
        "lesion_area_px": lesion_area_px,
        "leaf_area_cm2": leaf_area_cm2,
        "lesion_area_cm2": lesion_area_cm2,
        "severity_pct": severity_pct,
        "hull_area_px": hull_area,
        "hull_missing_pct": missing_pct_hull,
        "ellipse_area_px": ellipse_area_px,
        "ellipse_missing_pct": missing_pct_ellipse,
        "damage_pct_inpaint": damage_pct_inpaint,
        "leaf_mean_B": leaf_mean[0],
        "leaf_mean_G": leaf_mean[1],
        "leaf_mean_R": leaf_mean[2],
        "lesion_mean_B": lesion_mean[0],
        "lesion_mean_G": lesion_mean[1],
        "lesion_mean_R": lesion_mean[2]
    })

    # --- visualization ---
    overlay = inpainted_img.copy()
    overlay[leaf_mask_recon == 0] = (0, 0, 0)
    cv2.drawContours(overlay, [hull], -1, (255, 0, 0), 2)
    cv2.drawContours(overlay, cv2.findContours(lesion_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0], -1, (0, 0, 255), 2)
    cv2.putText(overlay, f"Severity: {severity_pct:.1f}% | Missing: {missing_pct_hull:.1f}% | Repaired: {damage_pct_inpaint:.1f}%", 
                (20,40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
    cv2.imshow("Leaf Analysis", overlay)
    cv2.waitKey(200)

cv2.destroyAllWindows()

# -------- SAVE --------
df = pd.DataFrame(records)
df.to_csv(output_csv, index=False)
print("✅ Tier 2 extraction (with inpainting) complete.")
print(df.head())
# ============================================================


✅ Tier 2 extraction (with inpainting) complete.
                       filename  leaf_area_px  lesion_area_px  leaf_area_cm2  \
0  20211008_124249 (Custom).jpg        200792           76080        20.0792   
1  20211008_124250 (Custom).jpg        188037           66566        18.8037   
2  20211008_124252 (Custom).jpg        211292           70074        21.1292   
3  20211008_124253 (Custom).jpg        189573           60201        18.9573   
4  20211008_124256 (Custom).jpg        250026          119754        25.0026   

   lesion_area_cm2  severity_pct  hull_area_px  hull_missing_pct  \
0           7.6080     37.889956      200208.5               0.0   
1           6.6566     35.400480      187702.0               0.0   
2           7.0074     33.164531      211133.5               0.0   
3           6.0201     31.756105      189406.0               0.0   
4          11.9754     47.896619      249767.0               0.0   

   ellipse_area_px  ellipse_missing_pct  damage_pct_inpaint  l

In [3]:
# ============================================================
# TIER 1 & 2: Complete Leaf Feature Extractor (with Inpainting)
# ============================================================

import cv2
import numpy as np
import pandas as pd
from pathlib import Path
from skimage.feature import graycomatrix, graycoprops
from skimage.color import rgb2gray

# -------- SETTINGS --------
image_dir = "../../database/dataset/Anthracnose/" 
output_csv = "leaf_features_complete.csv"
cm_per_pixel = 0.01   # calibrate using a known reference

# -------- UTILITIES --------
def area_cm2(pixels, cm_per_pixel):
    return pixels * (cm_per_pixel ** 2)

def segment_leaf(img_hsv):
    lower = np.array([20, 40, 40])
    upper = np.array([90, 255, 255])
    mask = cv2.inRange(img_hsv, lower, upper)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((5,5), np.uint8))
    return mask

def segment_lesion(img_hsv, leaf_mask):
    lower = np.array([5, 30, 0])
    upper = np.array([25, 255, 160])
    mask = cv2.inRange(img_hsv, lower, upper)
    mask = cv2.bitwise_and(mask, leaf_mask)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
    return mask

def reconstruct_leaf_inpaint(img, leaf_mask):
    """Use OpenCV inpainting to reconstruct missing/damaged leaf areas."""
    holes = cv2.bitwise_not(leaf_mask)
    holes = cv2.morphologyEx(holes, cv2.MORPH_OPEN, np.ones((5,5), np.uint8))
    
    contours, _ = cv2.findContours(leaf_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return leaf_mask, 0, img
    
    largest_contour = max(contours, key=cv2.contourArea)
    hull = cv2.convexHull(largest_contour)
    hull_mask = np.zeros_like(leaf_mask)
    cv2.drawContours(hull_mask, [hull], -1, 255, -1)
    inpaint_mask = cv2.bitwise_and(holes, hull_mask)

    missing_px = cv2.countNonZero(inpaint_mask)
    hull_area = cv2.countNonZero(hull_mask)
    damage_pct_inpaint = (missing_px / hull_area * 100) if hull_area > 0 else 0

    inpainted_img = cv2.inpaint(img, inpaint_mask, 5, cv2.INPAINT_TELEA)
    reconstructed_mask = cv2.bitwise_or(leaf_mask, inpaint_mask)

    return reconstructed_mask, damage_pct_inpaint, inpainted_img

def calculate_geometry_features(contour, leaf_area_px, hull_area):
    """Calculates solidity, circularity, and aspect ratio."""
    if cv2.contourArea(contour) == 0:
        return 0, 0, 0, 0, 0
    
    # 1. Solidity: Area / Convex Hull Area
    solidity = float(leaf_area_px) / hull_area if hull_area > 0 else 0

    # 2. Circularity: 4*pi*Area / Perimeter^2
    perimeter = cv2.arcLength(contour, True)
    circularity = (4 * np.pi * leaf_area_px) / (perimeter ** 2) if perimeter > 0 else 0

    # 3. Aspect Ratio: Width / Height of Bounding Box
    _, _, w, h = cv2.boundingRect(contour)
    aspect_ratio = float(w) / h if h > 0 else 0
    
    return solidity, circularity, aspect_ratio, perimeter, w, h

def calculate_vegetation_indices(img, mask):
    """Calculates NDVI and ExG using image BGR channels."""
    # Split BGR channels
    B, G, R = img[:,:,0].astype(float), img[:,:,1].astype(float), img[:,:,2].astype(float)
    
    # Normalize channels to 0-1 range (important for robust indices)
    sum_RGB = R + G + B
    r = np.divide(R, sum_RGB, out=np.zeros_like(R), where=sum_RGB!=0)
    g = np.divide(G, sum_RGB, out=np.zeros_like(G), where=sum_RGB!=0)
    
    # ExG (Excess Green) = 2G - R - B (often used with normalized channels: 2g - r - b)
    ExG_map = 2.0 * G - R - B
    
    # Simple NDVI for RGB: (G - R) / (G + R) (NDVI uses Near-Infrared, this is a proxy)
    sum_GR = G + R
    NDVI_proxy_map = np.divide(G - R, sum_GR, out=np.zeros_like(G), where=sum_GR!=0)
    
    # Apply mask and calculate mean values only for the leaf area
    ExG_mean = cv2.mean(ExG_map, mask=mask)[0]
    NDVI_mean = cv2.mean(NDVI_proxy_map, mask=mask)[0]
    
    return ExG_mean, NDVI_mean

def calculate_glcm_texture(img, mask):
    """Calculates GLCM texture features (Contrast, Dissimilarity) for the lesion area."""
    # Convert image to grayscale and scale to 8-bit for GLCM
    img_gray = (rgb2gray(img) * 255).astype(np.uint8)

    # Use a tight bounding box around the lesion mask to minimize computation
    y_coords, x_coords = np.where(mask == 255)
    if len(y_coords) == 0:
        return 0, 0

    y_min, y_max = np.min(y_coords), np.max(y_coords)
    x_min, x_max = np.min(x_coords), np.max(x_coords)
    
    lesion_patch = img_gray[y_min:y_max+1, x_min:x_max+1]
    lesion_mask_patch = mask[y_min:y_max+1, x_min:x_max+1]
    
    if lesion_patch.size == 0 or np.sum(lesion_mask_patch) == 0:
        return 0, 0
    
    # Apply a gentle blur to normalize noise before GLCM
    lesion_patch = cv2.GaussianBlur(lesion_patch, (3, 3), 0)
    
    # GLCM calculation (distance=1, angle=0 for simple texture)
    # GLCM works best on the lesion region's pixel values
    try:
        # Use only values within the mask to compute the matrix
        pixels_in_mask = lesion_patch[lesion_mask_patch > 0]
        
        # If there are not enough unique values, GLCM can fail.
        if len(np.unique(pixels_in_mask)) < 2:
            return 0, 0

        # Create a temporary image where everything outside the lesion is the average of the lesion
        temp_img = lesion_patch.copy()
        temp_img[lesion_mask_patch == 0] = np.mean(pixels_in_mask).astype(np.uint8)

        glcm = graycomatrix(temp_img, [1], [0], levels=256, symmetric=True, normed=True)
        
        contrast = graycoprops(glcm, 'contrast')[0, 0]
        dissimilarity = graycoprops(glcm, 'dissimilarity')[0, 0]
        
        return contrast, dissimilarity
    except ValueError:
        return 0, 0


# -------- MAIN LOOP --------
records = []

for path in Path(image_dir).glob("*.jpg"):
    img = cv2.imread(str(path))
    if img is None:
        continue

    # Resize for consistent feature comparison and speed
    img = cv2.resize(img, (800, int(img.shape[0]*800/img.shape[1])))
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # --- segmentation ---
    leaf_mask = segment_leaf(hsv)
    lesion_mask = segment_lesion(hsv, leaf_mask)

    # --- reconstruction (inpainting) ---
    leaf_mask_recon, damage_pct_inpaint, inpainted_img = reconstruct_leaf_inpaint(img, leaf_mask)

    # Find contours on reconstructed leaf and lesions
    leaf_contours, _ = cv2.findContours(leaf_mask_recon, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    lesion_contours, _ = cv2.findContours(lesion_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if not leaf_contours:
        continue
    largest_contour = max(leaf_contours, key=cv2.contourArea)

    # --- TIER 1 BASE METRICS ---
    leaf_area_px = cv2.countNonZero(leaf_mask_recon)
    lesion_area_px = cv2.countNonZero(lesion_mask)
    leaf_area_cm2 = area_cm2(leaf_area_px, cm_per_pixel)
    lesion_area_cm2 = area_cm2(lesion_area_px, cm_per_pixel)
    severity_pct = (lesion_area_px / leaf_area_px * 100) if leaf_area_px > 0 else 0
    lesion_count = len(lesion_contours) # TIER 2
    
    # --- TIER 1 Mean Lesion Size ---
    mean_lesion_size_px = (lesion_area_px / lesion_count) if lesion_count > 0 else 0

    # --- TIER 2 SHAPE METRICS ---
    hull = cv2.convexHull(largest_contour)
    hull_area = cv2.contourArea(hull)
    
    solidity, circularity, aspect_ratio, perimeter, bounding_w, bounding_h = \
        calculate_geometry_features(largest_contour, leaf_area_px, hull_area)
    
    # TIER 1 Color Stats
    leaf_mean = cv2.mean(img, mask=leaf_mask_recon)[:3]
    lesion_mean = cv2.mean(img, mask=lesion_mask)[:3]
    
    # TIER 2 Color Ratio
    # We use Green channel as it is most sensitive for stress in this color space
    leaf_G = leaf_mean[1]
    lesion_G = lesion_mean[1]
    color_ratio_G = (lesion_G / leaf_G) if leaf_G > 0 else 0
    
    # --- TIER 2 Vegetative Indices ---
    ExG_mean, NDVI_proxy_mean = calculate_vegetation_indices(img, leaf_mask_recon)

    # --- TIER 2 Texture Features (on Lesion) ---
    glcm_contrast, glcm_dissimilarity = calculate_glcm_texture(img, lesion_mask)


    # --- save record ---
    records.append({
        # CORE IDENTIFIERS
        "filename": path.name,

        # TIER 1: AREA & SEVERITY
        "leaf_area_cm2": leaf_area_cm2,
        "lesion_area_cm2": lesion_area_cm2,
        "severity_pct": severity_pct,
        
        # TIER 1 & 2: COLOR STATS
        "leaf_mean_B": leaf_mean[0],
        "leaf_mean_G": leaf_mean[1],
        "leaf_mean_R": leaf_mean[2],
        "lesion_mean_B": lesion_mean[0],
        "lesion_mean_G": lesion_mean[1],
        "lesion_mean_R": lesion_mean[2],
        "lesion_to_leaf_color_ratio_G": color_ratio_G,
        
        # TIER 2: INDICES
        "ExG_mean": ExG_mean,
        "NDVI_proxy_mean": NDVI_proxy_mean,
        
        # TIER 2: LESION COUNT/SIZE
        "lesion_count": lesion_count,
        "mean_lesion_size_px": mean_lesion_size_px,

        # TIER 2: SHAPE & RECONSTRUCTION
        "leaf_solidity": solidity,
        "leaf_circularity": circularity,
        "leaf_aspect_ratio": aspect_ratio,
        "damage_pct_inpaint": damage_pct_inpaint,
        
        # TIER 2: TEXTURE
        "lesion_glcm_contrast": glcm_contrast,
        "lesion_glcm_dissimilarity": glcm_dissimilarity,
    })

    # --- visualization (optional, for debugging) ---
    overlay = inpainted_img.copy()
    cv2.putText(overlay, f"Severity: {severity_pct:.1f}% | Solidity: {solidity:.2f} | Count: {lesion_count}", 
                (20,40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
    cv2.imshow("Complete Leaf Analysis", overlay)
    cv2.waitKey(1)

cv2.destroyAllWindows()

# -------- SAVE --------
df = pd.DataFrame(records)
df.to_csv(output_csv, index=False)
print("✅ Complete Tier 1 & 2 extraction (with inpainting) complete.")
print("\n--- Example Data Output ---\n")

# Utility function for the digestible report
def get_digestible_report(row):
    report = f"🌿 **Leaf Feature Report: {row['filename']}** 🌿\n"
    report += "--------------------------------------------------\n"
    
    # 1. Severity & Impact
    report += "## 📈 Health & Severity (Tier 1)\n"
    report += f"* **% Severity:** **{row['severity_pct']:.2f}%**\n"
    report += f"* **Lesion Area:** {row['lesion_area_cm2']:.2f} cm²\n"
    report += f"* **Leaf Area (Recon):** {row['leaf_area_cm2']:.2f} cm²\n"
    
    report += "\n## 🦠 Lesion Analysis (Tier 1 & 2)\n"
    report += f"* **Lesion Count:** {int(row['lesion_count'])}\n"
    report += f"* **Mean Lesion Size:** {row['mean_lesion_size_px']:.2f} px (Captures spread quality)\n"
    
    # 2. Stress/Health Indicators
    report += "\n## 🧪 Stress Indicators (Tier 2 Indices)\n"
    report += f"* **ExG (Excess Green):** {row['ExG_mean']:.2f} (Higher is greener/healthier)\n"
    report += f"* **NDVI Proxy:** {row['NDVI_proxy_mean']:.2f} (Health/Vigor index)\n"
    report += f"* **Lesion-Leaf Color Ratio (G):** {row['lesion_to_leaf_color_ratio_G']:.2f} (Stress intensity)\n"
    
    # 3. Shape & Damage Assessment
    report += "\n## 📐 Shape & Damage (Tier 2 Geometry)\n"
    report += f"* **Leaf Solidity:** {row['leaf_solidity']:.3f} (Deviation from convex hull. Lower = more holes/damage)\n"
    report += f"* **Circularity:** {row['leaf_circularity']:.3f} (How close to a circle. Ideal leaf $\approx 0.3-0.5$)\n"
    report += f"* **Aspect Ratio:** {row['leaf_aspect_ratio']:.2f} (Width/Height. Captures elongation)\n"
    report += f"* **Inpainting Repair %:** {row['damage_pct_inpaint']:.1f}% (Estimated area reconstructed)\n"
    
    # 4. Texture for ML
    report += "\n## ✨ Texture (ML Features)\n"
    report += f"* **GLCM Contrast:** {row['lesion_glcm_contrast']:.2f} (Local variation. High value = rough/varied lesion texture)\n"
    report += f"* **GLCM Dissimilarity:** {row['lesion_glcm_dissimilarity']:.2f} (Measures distance in intensity pairs)\n"
    
    return report

# --- Main loop and save section remains the same, but I added the call to the report function ---
if not df.empty:
    first_leaf = df.iloc[0]
    print(get_digestible_report(first_leaf))

✅ Complete Tier 1 & 2 extraction (with inpainting) complete.

--- Example Data Output ---

🌿 **Leaf Feature Report: 20211008_124249 (Custom).jpg** 🌿
--------------------------------------------------
## 📈 Health & Severity (Tier 1)
* **% Severity:** **37.89%**
* **Lesion Area:** 7.61 cm²
* **Leaf Area (Recon):** 20.08 cm²

## 🦠 Lesion Analysis (Tier 1 & 2)
* **Lesion Count:** 18
* **Mean Lesion Size:** 4226.67 px (Captures spread quality)

## 🧪 Stress Indicators (Tier 2 Indices)
* **ExG (Excess Green):** 51.86 (Higher is greener/healthier)
* **NDVI Proxy:** -0.03 (Health/Vigor index)
* **Lesion-Leaf Color Ratio (G):** 0.92 (Stress intensity)

## 📐 Shape & Damage (Tier 2 Geometry)
* **Leaf Solidity:** 1.003 (Deviation from convex hull. Lower = more holes/damage)
* **Circularity:** 0.521 (How close to a circle. Ideal leaf $pprox 0.3-0.5$)
* **Aspect Ratio:** 0.37 (Width/Height. Captures elongation)
* **Inpainting Repair %:** 6.1% (Estimated area reconstructed)

## ✨ Texture (ML Features

🌿 **Leaf Feature Report: 20211008_124249 (Custom).jpg** 🌿
--------------------------------------------------
## 📈 Health & Severity (Tier 1)
* **% Severity:** **37.89%**
* **Lesion Area:** 7.61 cm²
* **Leaf Area (Recon):** 20.08 cm²

## 🦠 Lesion Analysis (Tier 1 & 2)
* **Lesion Count:** 18
* **Mean Lesion Size:** 4226.67 px (Captures spread quality)

## 🧪 Stress Indicators (Tier 2 Indices)
* **ExG (Excess Green):** 51.86 (Higher is greener/healthier)
* **NDVI Proxy:** -0.03 (Health/Vigor index)
* **Lesion-Leaf Color Ratio (G):** 0.92 (Stress intensity)

## 📐 Shape & Damage (Tier 2 Geometry)
* **Leaf Solidity:** 1.003 (Deviation from convex hull. Lower = more holes/damage)
* **Circularity:** 0.521 (How close to a circle. Ideal leaf $pprox 0.3-0.5$)
* **Aspect Ratio:** 0.37 (Width/Height. Captures elongation)
* **Inpainting Repair %:** 6.1% (Estimated area reconstructed)
...
## ✨ Texture (ML Features)
* **GLCM Contrast:** 1.67 (Local variation. High value = rough/varied lesion texture)
* **GLCM Dissimilarity:** 0.28 (Measures distance in intensity pairs)

In [6]:
import pandas as pd
import numpy as np
import os

# --- FILE CONFIGURATION ---
INPUT_FILE = 'leaf_features_complete.csv'
OUTPUT_CSV = 'analysis_results.csv'
OUTPUT_REPORT = 'analysis_report.md'
# --------------------------

def analyze_health(row: pd.Series) -> pd.Series:
    """
    Applies expert-defined thresholds to the leaf feature data for a single row
    and generates actionable insights.
    """
    try:
        severity_pct = row['severity_pct']
        leaf_solidity = row['leaf_solidity']
        ndvi_proxy = row['NDVI_proxy_mean']
    except KeyError as e:
        print(f"Error: Missing expected column in CSV: {e}. Please check your CSV column names.")
        # Return default error state
        return pd.Series({
            'severity_class': 'ERROR',
            'action_plan': 'Check column names.',
            'solidity_status': 'ERROR',
            'vigor_status': 'ERROR'
        })


    # --- 1. Severity Classification (Primary Action Trigger) ---
    if severity_pct <= 1.0:
        severity_class = "Trace"
        action_plan = "MONITOR: Disease is present but negligible. Focus on cultural practices (air flow, dry foliage)."
    elif 1.0 < severity_pct <= 5.0:
        severity_class = "Low to Moderate"
        action_plan = "EARLY TREATMENT: Apply preventative fungicide, especially if weather favors disease. This is the economic action point."
    elif 5.0 < severity_pct <= 20.0:
        severity_class = "Moderate to High"
        action_plan = "URGENT TREATMENT: Fungicide application is NECESSARY to halt progression. Consider removing heavily infected leaves immediately."
    else: # > 20.0%
        severity_class = "Severe"
        action_plan = "SALVAGE & LONG-TERM: Focus on protecting new growth. Immediate sanitation (removing leaf debris) is critical for future seasons."

    # --- 2. Structural Integrity (Cultural Action Trigger) ---
    if leaf_solidity < 0.80:
        solidity_status = "CRITICAL Structural Loss (Needs Sanitation)"
    elif leaf_solidity < 0.90:
        solidity_status = "Moderate Irregularity (Structural Stress)"
    else:
        solidity_status = "Good Integrity"

    # --- 3. Plant Vigor (Nutritional Action Trigger) ---
    # Assuming 0.40 is the CRITICAL threshold for NDVI_proxy
    if ndvi_proxy < 0.40:
        vigor_status = "CRITICAL Vigor Loss (Immediate Nutrient Check)"
    elif ndvi_proxy < 0.45:
        vigor_status = "Low Vigor/High Stress (Potential Nutrient Deficiency)"
    else:
        vigor_status = "Adequate Vigor"

    # Compile the results into a Series to be appended back to the DataFrame
    return pd.Series({
        'severity_class': severity_class,
        'action_plan': action_plan,
        'solidity_status': solidity_status,
        'vigor_status': vigor_status,
        'final_recommendation': (
            f"**SEVERITY:** {severity_class} ({severity_pct:.2f}%) | "
            f"**VIGOR:** {vigor_status} | "
            f"**INTEGRITY:** {solidity_status}"
        )
    })

def generate_markdown_report(df: pd.DataFrame, output_file: str):
    """Generates a clean, human-readable summary report in Markdown format."""
    print(f"\n-> Generating Markdown Report: {output_file}")

    report_content = f"""
# Plant Disease Severity Analysis Report
**Generated:** {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}
**Total Leaves Analyzed:** {len(df)}

---

## 🚨 Critical Action Summary

The following table summarizes the immediate action required for each leaf based on the expert thresholds.

| Filename | Severity (%) | Severity Class | Primary Action Plan | Vigor Status | Integrity Status |
| :--- | :--- | :--- | :--- | :--- | :--- |
"""

    for index, row in df.iterrows():
        report_content += (
            f"| {row['filename']} "
            f"| **{row['severity_pct']:.2f}%** "
            f"| **{row['severity_class']}** "
            f"| {row['action_plan']} "
            f"| {row['vigor_status']} "
            f"| {row['solidity_status']} |\n"
        )

    report_content += "\n---\n\n"

    # Add the full data table to the report for completeness
    report_content += "## Detailed Feature Data\n"
    report_content += "*(Scroll right to view all 19 metrics plus the new analysis columns.)*\n\n"
    report_content += df.to_markdown(index=False)

    with open(output_file, 'w') as f:
        f.write(report_content)
    print(f"-> Report saved successfully to {output_file}")


def main():
    """Main function to orchestrate reading, analysis, and reporting."""
    if not os.path.exists(INPUT_FILE):
        print(f"FATAL ERROR: Input file '{INPUT_FILE}' not found.")
        print("Please ensure your CSV data is saved as 'leaf_data.csv' in the same directory.")
        return

    print(f"Reading data from: {INPUT_FILE}...")
    try:
        df = pd.read_csv(INPUT_FILE)
    except Exception as e:
        print(f"FATAL ERROR while reading CSV: {e}")
        return

    # Apply the analysis function to every row in the DataFrame
    # axis=1 means the function is applied row-wise
    analysis_df = df.apply(analyze_health, axis=1)

    # Merge the new analysis columns back into the main DataFrame
    df = pd.concat([df, analysis_df], axis=1)

    print(f"Successfully analyzed {len(df)} records.")

    # Save the full analysis (including original metrics and new columns) to CSV
    df.to_csv(OUTPUT_CSV, index=False)
    print(f"-> Full results saved to: {OUTPUT_CSV}")

    # Generate the concise Markdown report
    generate_markdown_report(df, OUTPUT_REPORT)
    print("\nProcessing complete!")
    print(f"Review the key actions in '{OUTPUT_REPORT}'.")

if __name__ == '__main__':
    main()


Reading data from: leaf_features_complete.csv...
Successfully analyzed 500 records.
-> Full results saved to: analysis_results.csv

-> Generating Markdown Report: analysis_report.md
-> Report saved successfully to analysis_report.md

Processing complete!
Review the key actions in 'analysis_report.md'.


In [ ]:
# ============================================================
# TIER 1 & 2: Complete Leaf Feature Extractor (with Inpainting)
# - MODIFIED FOR SINGLE IMAGE PROCESSING & CSV APPENDING -
# ============================================================

import cv2
import numpy as np
import pandas as pd
from pathlib import Path
from skimage.feature import graycomatrix, graycoprops
from skimage.color import rgb2gray

# -------- SETTINGS --------
# The output CSV file where results will be stored.
output_csv = "leaf_features_complete.csv" 
# Calibration factor: adjust this based on your camera setup.
cm_per_pixel = 0.01 

# -------- UTILITIES (Unchanged) --------
def area_cm2(pixels, cm_per_pixel):
    return pixels * (cm_per_pixel ** 2)

def segment_leaf(img_hsv):
    lower = np.array([20, 40, 40])
    upper = np.array([90, 255, 255])
    mask = cv2.inRange(img_hsv, lower, upper)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((5,5), np.uint8))
    return mask

def segment_lesion(img_hsv, leaf_mask):
    lower = np.array([5, 30, 0])
    upper = np.array([25, 255, 160])
    mask = cv2.inRange(img_hsv, lower, upper)
    mask = cv2.bitwise_and(mask, leaf_mask)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
    return mask

def reconstruct_leaf_inpaint(img, leaf_mask):
    """Use OpenCV inpainting to reconstruct missing/damaged leaf areas."""
    holes = cv2.bitwise_not(leaf_mask)
    holes = cv2.morphologyEx(holes, cv2.MORPH_OPEN, np.ones((5,5), np.uint8))
    
    contours, _ = cv2.findContours(leaf_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return leaf_mask, 0, img
    
    largest_contour = max(contours, key=cv2.contourArea)
    hull = cv2.convexHull(largest_contour)
    hull_mask = np.zeros_like(leaf_mask)
    cv2.drawContours(hull_mask, [hull], -1, 255, -1)
    inpaint_mask = cv2.bitwise_and(holes, hull_mask)

    missing_px = cv2.countNonZero(inpaint_mask)
    hull_area = cv2.countNonZero(hull_mask)
    damage_pct_inpaint = (missing_px / hull_area * 100) if hull_area > 0 else 0

    inpainted_img = cv2.inpaint(img, inpaint_mask, 5, cv2.INPAINT_TELEA)
    reconstructed_mask = cv2.bitwise_or(leaf_mask, inpaint_mask)

    return reconstructed_mask, damage_pct_inpaint, inpainted_img

def calculate_geometry_features(contour, leaf_area_px, hull_area):
    """Calculates solidity, circularity, and aspect ratio."""
    if cv2.contourArea(contour) == 0:
        return 0, 0, 0, 0, 0, 0
    
    solidity = float(leaf_area_px) / hull_area if hull_area > 0 else 0
    perimeter = cv2.arcLength(contour, True)
    circularity = (4 * np.pi * leaf_area_px) / (perimeter ** 2) if perimeter > 0 else 0
    _, _, w, h = cv2.boundingRect(contour)
    aspect_ratio = float(w) / h if h > 0 else 0
    
    return solidity, circularity, aspect_ratio, perimeter, w, h

def calculate_vegetation_indices(img, mask):
    """Calculates NDVI and ExG using image BGR channels."""
    B, G, R = img[:,:,0].astype(float), img[:,:,1].astype(float), img[:,:,2].astype(float)
    
    sum_RGB = R + G + B
    r = np.divide(R, sum_RGB, out=np.zeros_like(R), where=sum_RGB!=0)
    g = np.divide(G, sum_RGB, out=np.zeros_like(G), where=sum_RGB!=0)
    
    ExG_map = 2.0 * G - R - B
    sum_GR = G + R
    NDVI_proxy_map = np.divide(G - R, sum_GR, out=np.zeros_like(G), where=sum_GR!=0)
    
    ExG_mean = cv2.mean(ExG_map, mask=mask)[0]
    NDVI_mean = cv2.mean(NDVI_proxy_map, mask=mask)[0]
    
    return ExG_mean, NDVI_mean

def calculate_glcm_texture(img, mask):
    """Calculates GLCM texture features (Contrast, Dissimilarity) for the lesion area."""
    img_gray = (rgb2gray(img) * 255).astype(np.uint8)
    y_coords, x_coords = np.where(mask == 255)
    if len(y_coords) == 0:
        return 0, 0

    y_min, y_max = np.min(y_coords), np.max(y_coords)
    x_min, x_max = np.min(x_coords), np.max(x_coords)
    
    lesion_patch = img_gray[y_min:y_max+1, x_min:x_max+1]
    lesion_mask_patch = mask[y_min:y_max+1, x_min:x_max+1]
    
    if lesion_patch.size == 0 or np.sum(lesion_mask_patch) == 0:
        return 0, 0
    
    lesion_patch = cv2.GaussianBlur(lesion_patch, (3, 3), 0)
    
    try:
        pixels_in_mask = lesion_patch[lesion_mask_patch > 0]
        if len(np.unique(pixels_in_mask)) < 2:
            return 0, 0

        temp_img = lesion_patch.copy()
        temp_img[lesion_mask_patch == 0] = np.mean(pixels_in_mask).astype(np.uint8)
        glcm = graycomatrix(temp_img, [1], [0], levels=256, symmetric=True, normed=True)
        
        contrast = graycoprops(glcm, 'contrast')[0, 0]
        dissimilarity = graycoprops(glcm, 'dissimilarity')[0, 0]
        
        return contrast, dissimilarity
    except ValueError:
        return 0, 0

# -------- NEW: CORE PROCESSING FUNCTION --------
def process_image(image_path):
    """
    Processes a single leaf image and extracts all Tier 1 & 2 features.
    Returns a dictionary of features and the visualization image.
    """
    path = Path(image_path)
    img = cv2.imread(str(path))
    if img is None:
        print(f"Error: Could not read image at {image_path}")
        return None, None

    # --- Processing Steps (from original loop) ---
    img = cv2.resize(img, (800, int(img.shape[0]*800/img.shape[1])))
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    leaf_mask = segment_leaf(hsv)
    lesion_mask = segment_lesion(hsv, leaf_mask)
    leaf_mask_recon, damage_pct_inpaint, inpainted_img = reconstruct_leaf_inpaint(img, leaf_mask)

    leaf_contours, _ = cv2.findContours(leaf_mask_recon, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    lesion_contours, _ = cv2.findContours(lesion_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if not leaf_contours:
        print(f"Warning: No leaf contour found in {path.name}")
        return None, None
    largest_contour = max(leaf_contours, key=cv2.contourArea)

    # --- Feature Calculations ---
    leaf_area_px = cv2.countNonZero(leaf_mask_recon)
    lesion_area_px = cv2.countNonZero(lesion_mask)
    severity_pct = (lesion_area_px / leaf_area_px * 100) if leaf_area_px > 0 else 0
    lesion_count = len(lesion_contours)
    mean_lesion_size_px = (lesion_area_px / lesion_count) if lesion_count > 0 else 0
    
    hull = cv2.convexHull(largest_contour)
    hull_area = cv2.contourArea(hull)
    solidity, circularity, aspect_ratio, _, _, _ = calculate_geometry_features(largest_contour, leaf_area_px, hull_area)
    
    leaf_mean = cv2.mean(img, mask=leaf_mask_recon)[:3]
    lesion_mean = cv2.mean(img, mask=lesion_mask)[:3]
    color_ratio_G = (lesion_mean[1] / leaf_mean[1]) if leaf_mean[1] > 0 else 0
    
    ExG_mean, NDVI_proxy_mean = calculate_vegetation_indices(img, leaf_mask_recon)
    glcm_contrast, glcm_dissimilarity = calculate_glcm_texture(img, lesion_mask)

    # --- Create Feature Record ---
    record = {
        "filename": path.name,
        "leaf_area_cm2": area_cm2(leaf_area_px, cm_per_pixel),
        "lesion_area_cm2": area_cm2(lesion_area_px, cm_per_pixel),
        "severity_pct": severity_pct,
        "leaf_mean_B": leaf_mean[0], "leaf_mean_G": leaf_mean[1], "leaf_mean_R": leaf_mean[2],
        "lesion_mean_B": lesion_mean[0], "lesion_mean_G": lesion_mean[1], "lesion_mean_R": lesion_mean[2],
        "lesion_to_leaf_color_ratio_G": color_ratio_G,
        "ExG_mean": ExG_mean,
        "NDVI_proxy_mean": NDVI_proxy_mean,
        "lesion_count": lesion_count,
        "mean_lesion_size_px": mean_lesion_size_px,
        "leaf_solidity": solidity,
        "leaf_circularity": circularity,
        "leaf_aspect_ratio": aspect_ratio,
        "damage_pct_inpaint": damage_pct_inpaint,
        "lesion_glcm_contrast": glcm_contrast,
        "lesion_glcm_dissimilarity": glcm_dissimilarity,
    }
    
    # --- Create Visualization Image ---
    overlay = inpainted_img.copy()
    cv2.putText(overlay, f"Severity: {severity_pct:.1f}% | Solidity: {solidity:.2f} | Count: {lesion_count}", 
                (20,40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)

    return record, overlay

# -------- NEW: CSV SAVING FUNCTION --------
def append_record_to_csv(record, csv_filepath):
    """Appends a feature record to a CSV file. Creates the file and header if it doesn't exist."""
    file_exists = Path(csv_filepath).is_file()
    df = pd.DataFrame([record]) # Convert single record dict to a DataFrame
    
    # Use mode='a' to append; only write header if the file doesn't exist
    df.to_csv(csv_filepath, mode='a', header=not file_exists, index=False)
    print(f"✅ Successfully saved features for '{record['filename']}' to {csv_filepath}")

# -------- REPORTING FUNCTION (Unchanged) --------
def get_digestible_report(row):
    report = f"🌿 **Leaf Feature Report: {row['filename']}** 🌿\n"
    report += "--------------------------------------------------\n"
    report += "## 📈 Health & Severity (Tier 1)\n"
    report += f"* **% Severity:** **{row['severity_pct']:.2f}%**\n"
    report += f"* **Lesion Area:** {row['lesion_area_cm2']:.2f} cm²\n"
    report += f"* **Leaf Area (Recon):** {row['leaf_area_cm2']:.2f} cm²\n"
    report += "\n## 🦠 Lesion Analysis (Tier 1 & 2)\n"
    report += f"* **Lesion Count:** {int(row['lesion_count'])}\n"
    report += f"* **Mean Lesion Size:** {row['mean_lesion_size_px']:.2f} px (Captures spread quality)\n"
    report += "\n## 🧪 Stress Indicators (Tier 2 Indices)\n"
    report += f"* **ExG (Excess Green):** {row['ExG_mean']:.2f} (Higher is greener/healthier)\n"
    report += f"* **NDVI Proxy:** {row['NDVI_proxy_mean']:.2f} (Health/Vigor index)\n"
    report += f"* **Lesion-Leaf Color Ratio (G):** {row['lesion_to_leaf_color_ratio_G']:.2f} (Stress intensity)\n"
    report += "\n## 📐 Shape & Damage (Tier 2 Geometry)\n"
    report += f"* **Leaf Solidity:** {row['leaf_solidity']:.3f} (Deviation from convex hull. Lower = more holes/damage)\n"
    report += f"* **Circularity:** {row['leaf_circularity']:.3f} (How close to a circle. Ideal leaf $\approx 0.3-0.5$)\n"
    report += f"* **Aspect Ratio:** {row['leaf_aspect_ratio']:.2f} (Width/Height. Captures elongation)\n"
    report += f"* **Inpainting Repair %:** {row['damage_pct_inpaint']:.1f}% (Estimated area reconstructed)\n"
    report += "\n## ✨ Texture (ML Features)\n"
    report += f"* **GLCM Contrast:** {row['lesion_glcm_contrast']:.2f} (Local variation. High value = rough/varied lesion texture)\n"
    report += f"* **GLCM Dissimilarity:** {row['lesion_glcm_dissimilarity']:.2f} (Measures distance in intensity pairs)\n"
    return report

# -------- SCRIPT EXECUTION --------
if __name__ == "__main__":
    # ❗️ IMPORTANT: Change this path to the image you want to process
    input_image_path = "path/to/your/single_leaf_image.jpg" 

    # 1. Process the image to get features
    features_record, visual_output = process_image(input_image_path)
    
    # 2. If processing was successful, save and show results
    if features_record:
        # Save the extracted features to the CSV file
        append_record_to_csv(features_record, output_csv)
        
        # Print the detailed report to the console
        print("\n--- Digestible Report ---\n")
        print(get_digestible_report(features_record))

        # Show the visual result
        cv2.imshow("Complete Leaf Analysis", visual_output)
        print("\nPress any key to close the image window.")
        cv2.waitKey(0) # Waits indefinitely for a key press
        cv2.destroyAllWindows() 